[XTTS Huggingspace](https://huggingface.co/spaces/coqui/xtts)

[Online audio cutter](https://mp3cut.net/)

[Remove vocal from audio](https://vocalremover.org/https://vocalremover.org/)

[Enhance audio](https://podcast.adobe.com/enhance)

In [ ]:
base_path="/content"
# base_path="."

In [ ]:
#@title Install XTTS (It will take few minutes)

# https://huggingface.co/spaces/coqui/xtts
# https://docs.coqui.ai/en/dev/models/xtts.html

!pip install huggingface-hub
!rm -rf $base_path/TTS
!rm -rf $base_path/XTTS-v2

%cd $base_path
!git clone https://github.com/coqui-ai/TTS/

%cd $base_path/TTS
!make system-deps
!make install

%cd $base_path
# !git lfs install
# !git clone https://huggingface.co/coqui/XTTS-v2

from huggingface_hub import snapshot_download
snapshot_download(repo_id="coqui/XTTS-v2",local_dir=f"{base_path}/XTTS-v2")

!pip install numpy==1.26.2
!pip install tensorflow==2.12.0
!sudo apt install libcairo2-dev pkg-config python3-dev
!pip install pycairo
!pip install nltk
!pip install indic-nlp-library
!pip install deep_translator==1.11.4
!pip install gradio
!pip install pydub




from IPython.display import clear_output
clear_output()
#@title Restart Runtime
import time
time.sleep(6)
import os
os.kill(os.getpid(), 9)

In [ ]:
#@title #After disconnect run from Here

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
base_path='/content'
# base_path="."

In [ ]:
#@title Import Model
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import os
import shutil
import uuid
from nltk.tokenize import sent_tokenize
from indicnlp.tokenize.sentence_tokenize import sentence_split
import nltk
nltk.download('punkt')

def get_chunks(paragraph,Language, max_chars=100, further_chunks=True):
    global tokenizer_languages,languages

    # Map of language codes to NLTK and Indic NLP library language tokenizers

    lang=languages[Language]
    # Use Indic NLP for Hindi or NLTK's sentence tokenizer for other languages
    if lang == 'hi':
        sentences = sentence_split(paragraph, 'hi')
    else:
        nltk_lang = tokenizer_languages.get(lang, 'english')
        sentences = sent_tokenize(paragraph, language=nltk_lang)

    new_sentences = []
    i = 0

    while i < len(sentences):
        if len(sentences[i]) > max_chars:
            join_limit = 1
        else:
            join_limit = 1

        # Join sentences up to the join limit
        new_sentence = ' '.join(sentences[i:i + join_limit])

        # Further chunk if enabled and the new sentence exceeds max_chars
        if further_chunks and len(new_sentence) > max_chars:
            # Split at commas to create smaller chunks
            sub_sentences = new_sentence.split(',')
            temp_sentence = ''
            for j, part in enumerate(sub_sentences):
                # Adding comma back for readability except for the last segment in the split
                if j < len(sub_sentences) - 1:
                    part = part.strip() + ','
                else:
                    part = part.strip()  # No trailing comma for the last part

                # Check if adding the part exceeds max_chars
                if len(temp_sentence + part) > max_chars:
                    new_sentences.append(temp_sentence)
                    temp_sentence = part
                else:
                    temp_sentence += ' ' + part if temp_sentence else part

            # Add any remaining part as a sentence
            if temp_sentence:
                new_sentences.append(temp_sentence)
        else:
            new_sentences.append(new_sentence)

        # Increment by the dynamic join limit
        i += join_limit

    return new_sentences

languages = {
    'English': 'en',
    'Hindi': 'hi',
    'Arabic': 'ar',
    'Chinese': 'zh-cn',
    'Czech': 'cs',
    'Dutch': 'nl',
    'French': 'fr',
    'German': 'de',
    'Hungarian': 'hu',
    'Italian': 'it',
    'Japanese': 'ja',
    'Korean': 'ko',
    'Polish': 'pl',
    'Portuguese': 'pt',
    'Russian': 'ru',
    'Spanish': 'es',
    'Turkish': 'tr'
}

tokenizer_languages = {
    'en': 'english',
    'es': 'spanish',
    'fr': 'french',
    'de': 'german',
    'it': 'italian',
    'pt': 'portuguese',
    'pl': 'polish',
    'tr': 'turkish',
    'ru': 'russian',
    'nl': 'dutch',
    'cs': 'czech',
    'ar': 'arabic',
    'zh-cn': 'chinese',
    'ja': 'japanese',
    'hu': 'hungarian',
    'ko': 'korean',
    'hi': 'hindi'  # Indic NLP library will handle Hindi tokenization
}

from deep_translator import GoogleTranslator

def translate_text(text, Language):
    global languages
    target_language=languages[Language]
    if Language == "Chinese":
          target_language='zh-CN'
    translator = GoogleTranslator(target=target_language)
    translation = translator.translate(text.strip())
    t_text=str(translation)
    return t_text

def xtts(Enter_text,save_path,reference_voice_path,language,translate_text_flag):
  global languages
  if translate_text_flag==True:
    temp_text=translate_text(Enter_text, language)
    text=temp_text
  else:
    text=Enter_text
  tts.tts_to_file(text=text,
                file_path=save_path,
                speaker_wav=reference_voice_path,
                language=languages[language])
  return save_path

from pydub import AudioSegment

def merge_audio_files(file_paths, output_path):
    # Create an empty AudioSegment object to store the merged audio
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through the list of audio file paths and append each to the merged_audio
    for file_path in file_paths:
        audio = AudioSegment.from_file(file_path)
        merged_audio += audio

    # Export the merged audio to the specified output_path
    merged_audio.export(output_path, format="wav")


import re
def clean_file_name(file_path):
    # Get the base file name and extension
    file_name = os.path.basename(file_path)
    file_name, file_extension = os.path.splitext(file_name)

    # Replace non-alphanumeric characters with an underscore
    cleaned = re.sub(r'[^a-zA-Z\d]+', '_', file_name)

    # Remove any multiple underscores
    clean_file_name = re.sub(r'_+', '_', cleaned).strip('_')

    # Generate a random UUID for uniqueness
    random_uuid = uuid.uuid4().hex[:6]

    # Combine cleaned file name with the original extension
    clean_file_path = os.path.join(os.path.dirname(file_path), clean_file_name + f"_{random_uuid}" + file_extension)

    return clean_file_path
def get_new_tts_file_name(text):
    global temp_save_folder
    if text.endswith("."):
        text = text[:-1]
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    temp_name = f"{temp_save_folder}/{truncated_text}.wav"
    audio_file_name = clean_file_name(temp_name)
    return audio_file_name


from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def remove_silence(file_path):
    output_path=file_path.replace(".wav","_remove silence.wav")
    # Extract file name and format from the provided path
    file_name = os.path.basename(file_path)
    audio_format = "wav"

    # Reading and splitting the audio file into chunks
    sound = AudioSegment.from_file(file_path, format=audio_format)
    audio_chunks = split_on_silence(sound,
                                    min_silence_len=100,
                                    silence_thresh=-45,
                                    keep_silence=50)

    # Putting the file back together
    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk


    combined.export(output_path, format=audio_format)
    print("Remove Silence Done")
    return output_path


def voice_clone(input_text,reference_audio_path,language,translate_text_flag,remove_silence_flag):
  global temp_save_folder
  chunks_folder=f"{temp_save_folder}/temp_clone_tts/"
  if os.path.exists(chunks_folder):
    shutil.rmtree(chunks_folder)
  os.mkdir(chunks_folder)
  sentences=get_chunks(input_text,language, max_chars=100, further_chunks=True)
  clone_chunk=[]
  for index, text in enumerate(sentences):
    temp_save_path=f"{chunks_folder}/{index}.wav"
    xtts(text,temp_save_path,reference_audio_path,language,translate_text_flag)
    if os.path.exists(temp_save_path):
      clone_chunk.append(temp_save_path)
  save_path=get_new_tts_file_name(input_text)
  merge_audio_files(clone_chunk, save_path)
  if os.path.exists(save_path):
    if remove_silence_flag:
      output_path=remove_silence(save_path)
      return output_path
    else:
      return save_path
  else:
    return None

#import tts
%cd $base_path
from TTS.api import TTS
tts = TTS(model_path=f"{base_path}/XTTS-v2/", config_path=f"{base_path}/XTTS-v2/config.json", progress_bar=True, gpu=True)
temp_save_folder=f"{base_path}/clone_tts"
if not os.path.exists(temp_save_folder):
  os.mkdir(temp_save_folder)
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Run Gradio Interface
import gradio as gr
def gradio_call(Reference_Audio_Path,TTS_Text,Language,Auto_Translate,Remove_Silence):
  clone_audio_path=voice_clone(TTS_Text,Reference_Audio_Path,Language,Auto_Translate,Remove_Silence)
  return clone_audio_path





with gr.Blocks() as demo:
    # gr.Markdown(f"""# XTTS-V2 Voice Clone
    # [Online Audio Cutter](https://mp3cut.net/)
    # """)
    gr.Markdown(f"""<div style="text-align: center;">
    <h1><a href="https://huggingface.co/coqui/XTTS-v2" style="text-decoration: none; color: orange;">XTTS-V2 Voice Clone</a></h1>
    <a href="https://mp3cut.net/" style="text-decoration: none; color: blue;">Online Audio Cutter</a>
    </div>
    """)


    audio_file=gr.Audio(label="Reference Audio", type="filepath")
    input_text = gr.Textbox(label="Text to Generate", lines=6)
    language_name=gr.Dropdown(label="Language",choices=['English', 'Hindi', 'Arabic', 'Chinese', 'Czech', 'Dutch', 'French', 'German', 'Hungarian', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Spanish', 'Turkish'],value="English")
    generate_btn = gr.Button("Synthesize", variant="primary")
    with gr.Accordion("Advanced Settings", open=False):
        auto_translate=gr.Checkbox(value=True,label="Auto Translate")
        no_silence=gr.Checkbox(value=True,label="Remove Silence")
    audio_output  = gr.Audio(label="Generated Audio")
    generate_btn.click(
      gradio_call,
      inputs=[audio_file,input_text,language_name,auto_translate,no_silence],
      outputs=[audio_output],
  )
demo.launch(debug=False,share=True)

In [ ]:
#@title Upload 6s Audio clip

# import os
# from google.colab import files
# from IPython.display import clear_output
# import uuid
# def upload_audio():
#     upload_folder = f"{base_path}/user_upload"

#     os.makedirs(upload_folder, exist_ok=True)

#     os.chdir(upload_folder)

#     uploaded = files.upload()

#     os.chdir(base_path)

#     audio_name_list = []

#     for fn in uploaded.keys():
#         file_path = f"{upload_folder}/{fn}"
#         save_file_path = clean_file_name(file_path)

#         os.rename(file_path, save_file_path)
#         audio_name_list.append(save_file_path)

#     audio_extensions = ('.wav', '.mp3')
#     valid_audio_files = [f for f in audio_name_list if f.lower().endswith(audio_extensions)]

#     clear_output()

#     if valid_audio_files:
#         return valid_audio_files[0]
#     else:
#         print("Please upload an audio file.")
#         return None
# upload_audio()

'/content/user_upload/sanji_2_c51313.wav'

In [ ]:
#@title Run as funtion
# Text = 'Hi how are you'  # @param {type: "string"}
# Reference_Audio = '/content/a.mp3'  # @param {type: "string"}
# Language = "English" #@param ['English', 'Hindi', 'Arabic', 'Chinese', 'Czech', 'Dutch', 'French', 'German', 'Hungarian', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Spanish', 'Turkish']
# Auto_Translate = True  # @param {type: "boolean"}
# Remove_Silence= True  # @param {type: "boolean"}
# clone_audio_path=voice_clone(Text,Reference_Audio,Language,Auto_Translate,Remove_Silence)
# from IPython.display import Audio,display
# display(Audio(clone_audio_path, autoplay=False))

In [ ]:
# from google.colab import files
# files.download(clone_audio_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


[Audio Cutter](https://mp3cut.net/)